# Policy Gradient Methods 

## Yue Dong & Ali Emami
In this assignment, Ali and I experimented the policy gradient methods on Mountain car problem and easy21 (a simplified version of blackjack), as suggested in chapter 13 of the textbook. We first implemented the REINFORCE which is a policy gradient algorithm based on the complete return as in Monte Carlo algorithm. We compared the result of REINFORCE with or without the baseline. We then implemented the actor-critic methods with 1 or n step return. Lastly, we used actor-critic algorithm with eligibility trace and $\lambda$-return. 

## 1. Preliminaries

In policy gradient learning, instead of using $Q(s,a)$ to choose the best action in a certain state, we select an action based on the *learned parameterized policy* $\pi(a | s,\mathbb{\theta})$. In more details, the action a is choosen with the probability $\pi(a | s,\mathbb{\theta})$ given the state $s$ at time $t$ with the weight vector $\theta$.

Although selecting actions is not based on consulting a value function in policy gradient algorithm, sometimes, we still need a parameterized value function. For example, in REINFORCE, we need $V$ as the baseline and in actor-critic with 1-step or n-steps or $\lambda$-returns, we need $V$ to form the return.  We therefore could use function approximation to parameterize the value function with respect to the weights $w$. Thus, $\hat{V}=V(s,w)$.

### (a) policy approximation 
Since the action space is discrete in both mountain car and easy21, we from the parameterization of the policy with the preferences $h(s,a, \theta)$ which is similar to the case in multi-arm bandit. Then **the policy** is defined as an exponential softmax distribution (h bigger for $a$, $a$ is more likely to be chosen):
$$\pi(a|s,\theta)=\frac{exp(h(s,a,\theta))}{\sum_b exp(h(s,b,\theta))}, b\in A_s$$

Here we choose a linear function approxiamation in features to represent the **preferences**: 
$$h(s,a,\theta) = \theta^{T} \phi(s,a)$$ where **the features** $\phi(s,a) \in \mathbb{R}^n$ is constructed as follows:

#### (i) In easy21,  we define  $\phi(s,a)$ as a binary feature vector with $3*6*2 = 36$ features. 
Each binary feature
 has a value of 1 if (s,a) lies within the cuboid of state-space corresponding to
 that feature, and the action corresponding to that feature. The cuboids have
 the following overlapping intervals:
   - dealer(s) = [1: 4]; [4: 7]; [7: 10]
   - player(s) = [1: 6]; [4: 9]; [7: 12]; [10: 15]; [13: 18]; [16: 21]
   - a = 1 (hit); a = 0 (stick)

#### (ii) In mountain car,  we define  $\phi(s,a)$  as a binary feature vector obtained from tile coding with  (4∗9∗9)*3  features. 
we divide the 2D space into an 8x8 grid and then we offset it 3 times with 1/4 of a tile size to form 4 tilings.  We add one extra row and one extra column so that every point is covered by each tiling. A simple tile coding with offset is demonstraded as the following graph from Sutton's textbook. 
<img src="tile_coding.png" style="max-width:100%; width: 100%; max-width: none">

### Advantages of using policy parameterization over action-value parameterization:
-  the approximate policy can approach determinism rather than $\epsilon$ greedy algorithm.
-  useful when the policy is a simpler function to approximate than action-value function.

### The goal of policy learning here is to modify $\theta$ such that the performance measure $\eta(\theta)$ is maximized.
$$\theta_{t+1}=\theta_{t}+\alpha \nabla \eta  (\theta_t)$$

If we use $\eta(\theta)=v_{\pi_\theta}(s_0)$, then the gradient of $\eta(\theta)$ could be represented by 
<img src="policy_gradient_thm.png" style="max-width:50%; width: 50%; max-width: none">
Which is called **the policy gradient theorem**.

## 2. REINFORCE: Monte Carlo Policy Gradient
Using the policy gradient theorem, we could deduce that 
$$\nabla \eta(\theta) = E_{\pi}\big[\gamma^tG_t\frac{\nabla_{\theta}\pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}\big]$$
We sample $G_t$ on each time step and define the update as: 
<img src="reinforce.png" style="max-width:50%; width: 50%; max-width: none">
Since $G_t$ is the complete return from time $t$, REINFORCE is a Monte Carlo method.
<img src="reinforce_psedo.png" style="max-width:80%; width: 80%; max-width: none">
where $$\nabla_{\theta}log\pi(A_t|S_t,\theta) =  \frac{\nabla_{\theta}\pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}$$ 
and with linear action preferences, 
<img src="reinforce_update.png" style="max-width:50%; width: 50%; max-width: none">

In [10]:
import os
os.chdir("/Users/yuedong/Downloads/comp767_assignment_5/")
#%%

from easy21game import Easy21
from easy21feature import *
import numpy as np
from matplotlib import cm
#%%
env = Easy21()

In [11]:
# this block implements the policy approximation

def preference_cal(theta, state, action):
    h_s_a = np.dot(theta, phi(state,action))
    return h_s_a

def policy(theta, state, action):
    # there are 36 features
    h_s_a = preference_cal(theta, state, action)

In [ ]:
class Sarsa_Agent:
    def __init__(self, environment, mlambda, gamma=1, step_size=0.01):

        self.env = environment
        self.mlambda = mlambda
        self.gamma = gamma
        self.step_size = step_size
        #initialize everything
#        self.Q = np.zeros((self.env.dealer_space,
#                           self.env.player_space, 
#                           self.env.nA))
         
        self.V = np.zeros((self.env.dealer_space,
                           self.env.player_space))
        
        self.W = np.zeros(36)
        self.E = np.zeros(36)
        
        self.iterations = 0
        
        
    # Q is simply the dot product of phi and w
    def cal_Q(self,s,a):
        return np.dot(phi(s,a),self.W)


          # get optimal action based on ε-greedy exploration strategy  
    def epsilon_greedy_action(self, state, epsilon=0.05):
        # action = 0 stick
        # action = 1 hit
        hit = 1
        stick = 0
        # epsilon greedy policy
        if np.random.random() < epsilon:
            r_action = hit if np.random.random()<0.5 else stick
            return r_action
        else:
            action = np.argmax([self.cal_Q(state,0),self.cal_Q(state,1)])
            return action
        
    
    def train(self, iterations):        
        # Loop episodes
        for episode in range(iterations):
            self.E = np.zeros(36)

            # get initial state for current episode
            s = self.env._reset()
            a = self.epsilon_greedy_action(s)
            a_next = a
            term = False
            #r = 0
            
            # Execute until game ends
            while not term:
                
                # execute action
                s_next, r, term = self.env._step(a)[0:3]
                q = self.cal_Q(s,a)
                                
                if not term:
                    # choose next action with epsilon greedy policy
                    a_next = self.epsilon_greedy_action(s_next)
                    q_next = self.cal_Q(s_next,a_next)
                    delta = r + self.gamma * q_next - q
                else:
                    delta = r - q
                
                
                self.E =  self.gamma * self.mlambda * self.E +phi(s,a)
                self.W = self.W + self.step_size * delta * self.E

                # reassign s and a
                s = s_next
                a = a_next


        self.iterations += iterations

        # Derive value function
        for d in range(self.env.dealer_space):
            for p in range(self.env.player_space):
                self.V[d,p] = max([self.cal_Q((p+1,d+1),0),self.cal_Q((p+1,d+1),1)])
                
    def plot_frame(self, ax):
        def get_stat_val(x, y):
            return self.V[x, y]

        X = np.arange(0, self.env.dealer_space, 1)
        Y = np.arange(0, self.env.player_space, 1)
        X, Y = np.meshgrid(X, Y)
        Z = get_stat_val(X, Y)
        
        surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                               cmap=cm.coolwarm, linewidth=0, antialiased=False)
        return surf
#%%

agent = Sarsa_Agent(env, 0.5)
#%%
for i in range (1):
    agent.train(10000)

#print("learned values:",agent.V)
#agent.W

## 2. Mountain-Car task

### (1) mountain car environment
We first implemented the environment of the mountain car based on the example 8.2 (first version of sutton's book).

### (2) feature encoding with tile coding 
We encode the feature space of the mountaion car as 4 tilings of 8x8 grids with a shift of 1/4 of a tile size. We add one extra row and one extra column so that every point is covered by each tiling. Since we are doing control, the features $\phi(s,a)$ are defined based on states and actions.

In [33]:
import math
# this code is inspired from https://github.com/ctevans/.../Tilecoder.py

# mountain car has two variables: position(x-axis) and velocity
# -1.2 <= position <= 0.5
# -0.07 <= velocity <= 0.07

# devide the 2D space into an 8 by 8 grid
# then shift this grid with 1/4 of a tile width to obtain 4 tilings (partitions) 
numTiles = 8 
numTilings = 4

positionTileMove = ((0.5 + 1.2) / numTiles) / numTilings
velocityTileMove = ((0.07 + 0.07) / numTiles) /numTilings

# in order to make sure all points are covered after shifting 
# add one extra row and one extra column for shifting
# numver of total features  = 9x9x4
numFeatures = numTilings * (numTiles+1) * (numTiles+1)


# x = position, y = velocity
# note move a tiling by (a, b), then find the index of a point
# is the same as moving the points by (-a, -b)
# shift direction in this code is to the left-bottom corner
def tilecode(x,y,tileIndices):
    
    # find the distance of x to the leftmost position
    x = x + 1.2
    # find the distance of y to smallest velocity
    y = y + 0.07

    for i in range (numTilings):
        
        # in tiling i, we move a points by 
        # (-i*positionTileMove,i*velocityTileMove) for feature encoding
        xMove = i * (-positionTileMove)
        yMove = i * (-velocityTileMove)
	
        xTileIdx = int(numTiles * (x - xMove)/1.7)
        yTileIdx = int(numTiles * (y - yMove)/0.14)

        tileIndices[i] = i * 81 + ( yTileIdx * 9 + xTileIdx)
    
    
def tileCoderIndices(x,y):
    tileIndices = [-1]*numTilings
    tilecode(x,y,tileIndices)
    #print('Tile indices for input (%s,%s) are : '%(x,y), tileIndices)
    return tileIndices
                                                
#printTileCoderIndices(0.5,0.07)
#[-1, -1, -1, -1]
#Tile indices for input (0.5,0.07) are :  [80, 161, 242, 323]

# INPUTS
#   s: state =(position,velocity) 
#   a: action, integer: throttle reverse (-1), zero throttle (0), throttle forwards (1)
# returns a binary vector of length (4*9*9)*3 representing the features
def phi(s, a):
    tmp = np.zeros(shape=(4*9*9,3)) #zeros array of dim 3*6*2
    #putting one where a feature is on
    for i in tileCoderIndices(s[0],s[1]):
            tmp[i,a] = 1 
    return(tmp.flatten()) #returning 'vectorized' (1-dim) array

### (3) SARSA($\lambda$) with LA
We then modified SARSA($\lambda$) with LA for the mountain car problem.